In [2]:
import numpy as np
import pandas as pd

import pytz

import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation
from astroplan import Observer, FixedTarget

import warnings
warnings.filterwarnings('ignore', category=Warning)


from astroplan.plots import plot_sky, plot_airmass
from astroplan import Observer, FixedTarget, time_grid_from_range, observability_table, moon_illumination
from astroplan import AirmassConstraint, MoonSeparationConstraint


In [3]:
catalog = pd.read_csv('./Data/galaxies.csv', skiprows=58, sep=';', usecols=[2,3,4], names=['Name', 'RA', 'Dec'])
new = catalog[204:220]
new

,Name,RA,Dec
204,WKK 4098,14 35 57.25,-51 55 05.5
205,2MASX J14381221-7239480,14 38 08.90,-72 39 21.0
206,2MASX J14382229-7240149,14 38 22.25,-72 40 12.0
207,WKK 4172,14 42 08.55,-48 53 07.5
208,WKK 4260,14 45 40.10,-49 20 54.5
209,WKK 4266,14 45 49.25,-49 22 48.0
210,WKK 4551,15 01 13.80,-47 59 11.0
211,2MASX J15035031-7729407,15 03 49.75,-77 29 40.0
212,ESO 067-G005,15 04 35.35,-72 36 56.5
213,WKK 4603,15 05 29.10,-49 34 07.0


In [15]:
newnew = catalog[204:214]
newnew

,Name,RA,Dec
204,WKK 4098,14 35 57.25,-51 55 05.5
205,2MASX J14381221-7239480,14 38 08.90,-72 39 21.0
206,2MASX J14382229-7240149,14 38 22.25,-72 40 12.0
207,WKK 4172,14 42 08.55,-48 53 07.5
208,WKK 4260,14 45 40.10,-49 20 54.5
209,WKK 4266,14 45 49.25,-49 22 48.0
210,WKK 4551,15 01 13.80,-47 59 11.0
211,2MASX J15035031-7729407,15 03 49.75,-77 29 40.0
212,ESO 067-G005,15 04 35.35,-72 36 56.5
213,WKK 4603,15 05 29.10,-49 34 07.0


In [16]:
## PLEASE NOTE THAT THIS WAS PREVIOUS CODE FROM BEFORE THAT IS NO LONGER NEEDED. 
# We used this when dealing with a previous catalog.

# dropping null value columns to avoid errors 
#catalog.dropna(inplace = True) 
  
# new data frame with split value columns 
#new = catalog["RA Dec"].str.split(" ", n = 1, expand = True) 
  
# making seperate first name column from new data frame 
#catalog["RA"]= new[0].astype(float)/15 
  
# making seperate last name column from new data frame 
#catalog["Dec"]= new[1] 
  
# Dropping old Name columns 
#good_catalog = catalog.drop(columns =["RA Dec"]) 
  
# df display 
#catalog.to_csv('./Data/good_galaxies.csv')

#target_table = pd.read_csv('./Data/good_catalog.csv', index_col=0)
#target_table = pd.read_csv('./Data/good_galaxies.csv', index_col=0)

#our_targets = target_table.query('(RA > 14) & (RA < 15)') #this is the range of RA from May 1-May 15
#our_targets

In [9]:
may_1 = Time("2019-05-01 12:00:00")
may_15 =Time("2019-05-15 12:00:00")


midnight_CTIO_1 = CTIO.midnight(may_1, which='next')
midnight_CTIO_15 = CTIO.midnight(may_15, which='next')

# Local Siderial Time (LST) will tell you the Right Ascension on the meridian at a specific time.


In [11]:
CTIO.local_sidereal_time(midnight_CTIO_1)


<Longitude 14.58108482 hourangle>

In [12]:
CTIO.local_sidereal_time(midnight_CTIO_15)

<Longitude 15.4884886 hourangle>

In [4]:
CTIO = Observer(longitude = 70.815 * u.deg,
                      latitude = -30.16527778 * u.deg,
                       elevation = 2788 * u.m,
                        timezone = 'America/Santiago',
                       name = "Cerro Tololo Interamerican Observatory"
                      )



In [18]:
table_values = newnew.as_matrix()
table_values

array([['WKK 4098               ', '14 35 57.25', '-51 55 05.5'],
       ['2MASX J14381221-7239480', '14 38 08.90', '-72 39 21.0'],
       ['2MASX J14382229-7240149', '14 38 22.25', '-72 40 12.0'],
       ['WKK 4172               ', '14 42 08.55', '-48 53 07.5'],
       ['WKK 4260               ', '14 45 40.10', '-49 20 54.5'],
       ['WKK 4266               ', '14 45 49.25', '-49 22 48.0'],
       ['WKK 4551               ', '15 01 13.80', '-47 59 11.0'],
       ['2MASX J15035031-7729407', '15 03 49.75', '-77 29 40.0'],
       ['ESO 067-G005           ', '15 04 35.35', '-72 36 56.5'],
       ['WKK 4603               ', '15 05 29.10', '-49 34 07.0']],
      dtype=object)

In [19]:
targets = [FixedTarget(coord=SkyCoord(ra = RA, dec = Dec, unit=(u.hourangle, u.deg)), name=Name)
           for Name, RA, Dec in table_values]

constraints = [AirmassConstraint(1.8)]

window_start = Time("2019-05-01 12:00:00")
window_end = Time("2019-05-15 12:00:00")

observe_start = CTIO.twilight_evening_astronomical(window_start, which='nearest')
observe_end = CTIO.twilight_morning_astronomical(window_start, which='next')
    
observing_range = [observe_start, observe_end]
observing_table = observability_table(constraints, CTIO, targets, time_range=observing_range)
print(observing_table)

      target name       ever observable ... fraction of time observable
----------------------- --------------- ... ---------------------------
WKK 4098                           True ...          0.9523809523809523
2MASX J14381221-7239480            True ...          0.9523809523809523
2MASX J14382229-7240149            True ...          0.9523809523809523
WKK 4172                           True ...          0.9047619047619048
WKK 4260                           True ...          0.9047619047619048
WKK 4266                           True ...          0.9047619047619048
WKK 4551                           True ...          0.9047619047619048
2MASX J15035031-7729407            True ...          0.8571428571428571
ESO 067-G005                       True ...          0.9047619047619048
WKK 4603                           True ...          0.9047619047619048
